<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Creating and Provisioning Storage with Python

## Azure Actions Covered

* Creating storage resources


In this lecture, we're going to take a look at how to create storage resources via the Python SDK.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). We'll add our new storage import as well.

In [3]:
# For generating storage account name
import random

from azure.identity import AzureCliCredential
from azure.mgmt.resource import ResourceManagementClient
# New import
from azure.mgmt.storage import StorageManagementClient

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

In [2]:
credential = AzureCliCredential()
resource_client = ResourceManagementClient(credential, AZURE_SUBSCRIPTION_ID)

In [4]:
if not resource_client.resource_groups.get(DEFAULT_RESOURCE_GROUP):
    rg_result = resource_client.resource_groups.create_or_update(
        DEFAULT_RESOURCE_GROUP,
        {'location': DEFAULT_LOCATION}
    )

In [5]:
storage_client = StorageManagementClient(credential, AZURE_SUBSCRIPTION_ID)

testes

In [15]:
STORAGE_ACCOUNT_BASE = 'benbstorage'
while True:
    account_name = f'{STORAGE_ACCOUNT_BASE}{random.randint(1,1000)}'
    if not storage_client.storage_accounts.check_name_availability({'name':account_name}).name_available:
        print(f'Account name {account_name} not available, trying again')
    else:
        print(f'Account name {account_name} available, creating account')
        poller =  storage_client.storage_accounts.begin_create(
            DEFAULT_RESOURCE_GROUP,
            account_name,
            {
                'location': DEFAULT_LOCATION,
                'kind': 'StorageV2',
                'sku': {'name': 'Standard_LRS'}
            }
        )
        poller_result = poller.result()
        print(f'{poller_result.name} has been provisioned')
        break

Account name benbstorage226 available, creating account
benbstorage226 has been provisioned


In [17]:
CONTAINER_NAME = 'bens-blob-container'
container = storage_client.blob_containers.create(
    DEFAULT_RESOURCE_GROUP,
    account_name,
    CONTAINER_NAME,
    {}
)

In [19]:
container.name

'bens-blob-container'